In [ ]:
# begin importing of standard modules
import re
import random
import os
import tkinter as tk
from tkinter import messagebox
import pandas as pd
import pprint as pp
import sys
# end importing of standard modules

# begin setting of home path variable and adding data directory to path import list
if 'Step_1.ipynb' or 'Step_1.py' or 'Step_1.exe' in os.listdir():
    home = os.getcwd()     
sys.path.append(os.path.join(home, 'data')) #This will add the data folder to the path variable so modules can be imported 
# end setting of home path variable and adding data directory to path import list

# begin creation Regular Expression that will check that a folder has been sanitized
validHash = re.compile(r'#\d{0,3}')
# end creation Regular Expression that will check that a folder has been sanitized

# begin import of created modules
try:   
    import ds
    import mypath
except ModuleNotFoundError:
   pass
# begin import of created modules

# begin importing and creation of a data frame
os.chdir(mypath.data)
df = pd.read_csv('data.csv')
# end importing and creation of a data frame

pd.set_option('max_colwidth', 200)
df